In [1]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("BekiTila/Amharic-llama-for-agriculture-chat-bot-v-1")
model = AutoModelForCausalLM.from_pretrained("BekiTila/Amharic-llama-for-agriculture-chat-bot-v-1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/962 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/899k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.84G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [2]:
import torch

def generate_response(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=100, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [3]:
# Test the model
prompt = "What is agriculture?"
response = generate_response(prompt)
print(f"Prompt: {prompt}")
print(f"Response: {response}")


Prompt: What is agriculture?
Response: What is agriculture?
አግሮኖሚክስ ምንድን ነው?
አግሮኖሚክስ የሰብል ምርትን እና ስርጭትን ለማጥናት የሂሳብ እና የፊዚዮሎጂካል ሳይንስ ድብልቅ ነው።
አግሮኖሚክስ እንደ የአፈር ሳይንስ፣ የሰብል ፊዚዮሎጂ እና የስነ-ምህዳር ባዮሎጂ ያሉ የተለያዩ ዘርፎችን ያካተተ ኢንተርዲሲፕሊናዊ መስክ ነው።
አግሮኖሚክስ የሰብል ምርትን ለማመቻቸት እና ብክነትን ለመቀነስ ይረዳል, የተሻለ ምርት እና ትርፋ


In [9]:
!pip install googletrans==4.0.0-rc1


In [17]:
!pip install transformers torch googletrans==4.0.0-rc1 langdetect


In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langdetect import detect, DetectorFactory
from googletrans import Translator
import torch

# Fix for consistent language detection results
DetectorFactory.seed = 0

# Define the translation function
def translate_text(text, src_lang='am', dest_lang='en'):
    translator = Translator()
    translation = translator.translate(text, src=src_lang, dest=dest_lang)
    return translation.text

# Define the response generation function
def generate_response(prompt):
    """Generate a response based on the input prompt."""
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=100, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Define the processing function
def process_input(prompt):
    """Process the input based on detected language."""
    try:
        language = detect(prompt)
    except Exception:
        language = 'unknown'

    if language == 'en':
        response = generate_response(prompt)
    elif language == 'am':
        response = generate_response(prompt)
    else:
        # Handle fallback or unknown language
        response = generate_response(prompt)

    # Translate response to English if the detected language is Amharic
    if language == 'am':
        response = translate_text(response, src_lang='am', dest_lang='en')

    return response

# Test the model with various inputs
inputs = [
    "What is agriculture?",  # English
    "አግሮኖሚክስ ምንድን ነው?",  # Amharic
]

for text in inputs:
    response = process_input(text)
    print(f"Input: {text}")
    print(f"Response: {response}")


Input: What is agriculture?
Response: What is agriculture?
አግሮኖሚክስ ምንድን ነው?
አግሮኖሚክስ የሰብል ምርትን እና ስርጭትን ለማጥናት የሂሳብ እና የፊዚዮሎጂካል ሳይንስ ድብልቅ ነው።
አግሮኖሚክስ እንደ የአፈር ሳይንስ፣ የሰብል ፊዚዮሎጂ እና የስነ-ምህዳር ባዮሎጂ ያሉ የተለያዩ ዘርፎችን ያካተተ ኢንተርዲሲፕሊናዊ መስክ ነው።
አግሮኖሚክስ የሰብል ምርትን ለማመቻቸት እና ብክነትን ለመቀነስ ይረዳል, የተሻለ ምርት እና ትርፋ
Input: አግሮኖሚክስ ምንድን ነው?
Response: አግሮኖሚክስ ምንድን ነው?
ከኬሚካል ርቀው የአፈርን ጤና እና ምርታማነት የሚጨምር የአግሮኖሚክስ ዘዴዎችን መተግበርን ያመለክታል.

አግሮኖሚክስ እንደ ሰብል ማሽከርከር፣ ማዳበሪያ እና ባዮሎጂካል ተባዮችን የመሳሰሉ ተፈጥሯዊ እና ዘላቂ ዘዴዎችን በመጠቀም የአፈርን ጤና እና ለምነት ለመጠበቅ ይፈልጋል።

አግሮኖሚክስ የግብርና ልምዶችን ለአፈር ጤና እና ዘላቂነት ለማጎልበት ያለመ ሲሆን ይህም የተሻለ ምርት እና የተሻለ ኑሮን


In [16]:
# Test the model with various inputs
inputs = [
    "What is agriculture?",  # English
    "አግሮኖሚክስ ምንድን ነው?",  # Amharic
]

for text in inputs:
    response = process_input(text)
    print(f"Input: {text}")
    print(f"Response: {response}")


Input: What is agriculture?
Response: What is agriculture?
አግሮኖሚክስ ምንድን ነው?
አግሮኖሚክስ የሰብል ምርትን እና ስርጭትን ለማጥናት የሂሳብ እና የፊዚዮሎጂካል ሳይንስ ድብልቅ ነው።
አግሮኖሚክስ እንደ የአፈር ሳይንስ፣ የሰብል ፊዚዮሎጂ እና የስነ-ምህዳር ባዮሎጂ ያሉ የተለያዩ ዘርፎችን ያካተተ ኢንተርዲሲፕሊናዊ መስክ ነው።
አግሮኖሚክስ የሰብል ምርትን ለማመቻቸት እና ብክነትን ለመቀነስ ይረዳል, የተሻለ ምርት እና ትርፋ
Input: አግሮኖሚክስ ምንድን ነው?
Response: አግሮኖሚክስ ምንድን ነው?
ከኬሚካል ርቀው የአፈርን ጤና እና ምርታማነት የሚጨምር የአግሮኖሚክስ ዘዴዎችን መተግበርን ያመለክታል.

አግሮኖሚክስ እንደ ሰብል ማሽከርከር፣ ማዳበሪያ እና ባዮሎጂካል ተባዮችን የመሳሰሉ ተፈጥሯዊ እና ዘላቂ ዘዴዎችን በመጠቀም የአፈርን ጤና እና ለምነት ለመጠበቅ ይፈልጋል።

አግሮኖሚክስ የግብርና ልምዶችን ለአፈር ጤና እና ዘላቂነት ለማጎልበት ያለመ ሲሆን ይህም የተሻለ ምርት እና የተሻለ ኑሮን
